# Simulated Method of Moments

Based on McFadden (*Econometrica*, 1989), the Simulated Method of Moments (SMM) is a method to estimate the parameters of a model by matching the moments of the data with the moments of the model. The moments are calculated by simulating the model many times and averaging the results. The method is particularly useful when the model is too complex to be solved analytically.

The SMM estimator, $\hat{\theta}_{S,T}(W)$, is define as:

$$
\hat{\theta}_{S,T}(W)= argmin_{\theta} \left[ \sum_{t=1}^{T}\left(\mu(x_t) - \frac{1}{S} \sum_{s=1}^{S} \mu(x(u^s_t, \theta)) \right) \right]^{'} W_T^{-1} \left[ \sum_{t=1}^{T}\left(\mu(x_t) - \frac{1}{S} \sum_{s=1}^{S} \mu(x(u^s_t, \theta)) \right) \right]
$$

where:
* $x_t$ are a sequence of $T$ observations of the data
* $\mu(\cdot)$ is a function of the data
* $x(u^s_t, \theta)$ is the model simulated data for the $s$-th simulation
* $W_T$ is a weighting matrix

## SMM with the McCall model

We'll use SMM to estimate two parameters of our job search model. Recall that the McCall model had four parameters:
1. $\theta$, the replacement rate for unemployment benefits
2. $\beta$, the discount factor
3. $\mu$, the wage offer distribution mean
4. $\sigma$, the wage offer distribution standard deviation

We'll assume that $\beta=0.99$ and we'll consider the problem of college graduates, for whom the replacement rate is $\theta=0.0$. We'll estimate the mean and standard deviation of the wage offer distribution.

## The data

For data, we'll use the following moments: the fraction of graduates who are employed after 90 days and the mean wage of those who are employed.  For [graduates from my business school who major in economics, finance, and marketing the numbers are](https://www.sc.edu/study/colleges_schools/moore/talent_recruiting/office_of_career_management/company_recruiter_resources/employment_statistics/index.php):

### Data Moments

|Major| Fraction Employed in 90 Days| Mean Salary |
|---|---|---|
|Economics| 86% | $72,542 |
|Accounting| 90% | $70,223 |
|Marketing| 85% | $63,249 |
|Real Estate| 78% | $70,867 |

## Estimation

We'll use an identify matrix as the weight matrix.  We'll need to estimate $\mu$ and $\sigma$ separately for each major. As such, we'll use [Dask](https://www.dask.org) to parallelize the estimation process.


In [96]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask
from dask import delayed
from dask.distributed import Client
import scipy.optimize as opt
import scipy.stats as stats
import plotly.express as px

## Step 1: Write a function to solve the McCall model

In [97]:
# Write the function to solve the McCall model


## Step 2: Write functions to calculate the moments

In [98]:
# Write a function for the 1st moment: fraction accepting offer by 90 days


In [99]:
# Write a function for the 2nd moment: mean accepted offer

## Step 3: Write a function represent the statistical objective function

This will be the moment error function that we want to minimize.

In [100]:
# Write the statistical objective function


## Step 4: Write the function you'll pass to the optimizer

This function will take the parameters and data moments and return the moment errors.

In [101]:
# Write the SMM estimation function


## Step 5: Write a function to estimate the parameters

This function will take the data moments and return the estimated parameters as well as the model moments at those parameter estimates.

In [102]:
# Main function to run the SMM estimation


## Step 6: Do the estimation -- for one major

In [ ]:
# Do estimation


## Step 7: Do the estimation for all majors

In this case, you'll want to loop over all the majors and estimate the parameters for each one.  

Note that we can use `dask.delayed` to parallelize the estimation process since each major is estimated independently.

In [ ]:
# Now let's do this for each major.  We can estimate each problem separately.
# Set up the dictionary of data moments:
data_moments_dict = {
    "Economics": np.array([0.86, 72_542 / 365]),
    "Accounting": np.array([0.90, 70_223 / 365]),
    "Marketing": np.array([0.85, 63_249 / 365]),
    "Real Estate": np.array([0.78, 70_867 / 365])
}


In [105]:
# put results in a dataframe
results_df = pd.DataFrame(estimation_results_dict).T

In [ ]:
# Plot mu, by major
# put mean into dollars per year
results_df["mu"] = np.exp(results_df["mu"].astype(float)) * 365
px.bar(results_df, x=results_df.index, y="mu", title="Estimated Mean Salary Offer, by Major")

In [ ]:
# Plot how std dev of salary offers varies by major
px.bar(results_df, x=results_df.index, y="sigma", title="Estimated Std Dev of Salary Offers, by Major")

In [ ]:
# Compare model to data moments in table
results_df[["model_moments", "data_moments"]]